In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
model_category = 'Men Tshirts'

In [ ]:
path_swin_v2 = "../../Models/Men Tshirts_swin_v2_preproc.pth"
path_vit_l16 = "../../Models/effnet_Men_Tshirts_vit_l_16.pth"

In [ ]:
train_df_Men_Tshirts = pd.read_csv('../../Preprocessor-FillNA/output/train_fillna_Men_Tshirts_effnet_b5_4epochs.csv')
train_df_Men_Tshirts

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,image_path
0,0,Men Tshirts,5,default,round,printed,default,short sleeves,/kaggle/input/visual-taxonomy/train_images/000...
1,1,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,/kaggle/input/visual-taxonomy/train_images/000...
2,2,Men Tshirts,5,default,polo,solid,solid,short sleeves,/kaggle/input/visual-taxonomy/train_images/000...
3,3,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,/kaggle/input/visual-taxonomy/train_images/000...
4,4,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,/kaggle/input/visual-taxonomy/train_images/000...
...,...,...,...,...,...,...,...,...,...
7262,7424,Men Tshirts,5,black,round,printed,default,short sleeves,/kaggle/input/visual-taxonomy/train_images/007...
7263,7425,Men Tshirts,5,default,round,printed,default,long sleeves,/kaggle/input/visual-taxonomy/train_images/007...
7264,7426,Men Tshirts,5,white,round,printed,default,short sleeves,/kaggle/input/visual-taxonomy/train_images/007...
7265,7429,Men Tshirts,5,black,round,printed,default,short sleeves,/kaggle/input/visual-taxonomy/train_images/007...


In [5]:
attr_columns = train_df_Men_Tshirts.filter(like='attr_').columns.to_list() # Adjust if more attributes exist
print(attr_columns)

['attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5']


In [6]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder for each attribute column
label_encoders = {}
for column in attr_columns:
    le = LabelEncoder()
    train_df_Men_Tshirts[column] = le.fit_transform(train_df_Men_Tshirts[column])
    label_encoders[column] = le  # Store the encoder for inverse transformation later if needed

# Check the updated DataFrame
# train_df_Men_Tshirts = train_df_Men_Tshirts.head()

In [7]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets
train_df1, val_df1 = train_test_split(train_df_Men_Tshirts, test_size=0.15, random_state=42)

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from PIL import Image

# Define custom dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, is_test=False):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test  # Flag to indicate if it's test set without labels

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.img_dir + self.dataframe.iloc[idx]['image_path']
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        if self.is_test:  # For test set, just return the image without labels
            return image
        
        # For train/validation set, return image and labels
        labels = self.dataframe.iloc[idx][attr_columns].values
        labels = labels.astype(np.int64)  # Ensure labels are integers
        labels = torch.tensor(labels, dtype=torch.long)
        
        return image, labels
    
    
# Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Apply Color Jitter
    transforms.RandomHorizontalFlip(),  # Apply Horizontal Flip with 50% probability
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
    
train_dataset = CustomDataset(dataframe=train_df1, img_dir='', transform=transform)
val_dataset = CustomDataset(dataframe=val_df1, img_dir='', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [9]:
import ssl

ssl._create_default_https_context = ssl._create_stdlib_context

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

class MultiOutputModelSwin(nn.Module):
    def __init__(self, num_classes):
        super(MultiOutputModelSwin, self).__init__()
        
        # Load a pretrained Swin V2 model
        self.base_model = models.swin_v2_b(weights="IMAGENET1K_V1")
        
        # Remove the original classification head
        in_features = self.base_model.head.in_features  # Get the input features for the head
        self.base_model.head = nn.Identity()  # Replace with identity to extract embeddings
        
        # Define output layers for each attribute dynamically
        self.output_layers = nn.ModuleDict()
        for attr, n_classes in num_classes.items():
            self.output_layers[attr] = nn.Linear(in_features, n_classes)  # Use `in_features` for each linear layer

    def forward(self, x):
        # Pass input through the Swin model to get embeddings
        x = self.base_model(x)
        
        # Compute output for each attribute
        outputs = {attr: layer(x) for attr, layer in self.output_layers.items()}
        
        return list(outputs.values())

# Example usage
# Define number of classes for each attribute
num_classes = {attr: train_df_Men_Tshirts[attr].nunique() for attr in attr_columns}

# Initialize the model
model_swin_v2 = MultiOutputModelSwin(num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_swin_v2.parameters(), lr=0.0001)

Downloading: "https://download.pytorch.org/models/swin_v2_b-781e5279.pth" to /root/.cache/torch/hub/checkpoints/swin_v2_b-781e5279.pth
100%|██████████| 336M/336M [00:01<00:00, 203MB/s]  


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

class MultiOutputModelVit(nn.Module):
    def __init__(self, num_classes):
        super(MultiOutputModelVit, self).__init__()
        
        # Load a pretrained ViT model
        self.base_model = models.vit_l_16(pretrained=True)
        
        # Remove the original classification head
        self.base_model.heads = nn.Identity()  # Replace with identity to extract embeddings
        
        # Define output layers for each attribute dynamically
        self.output_layers = nn.ModuleDict()
        for attr, n_classes in num_classes.items():
            self.output_layers[attr] = nn.Linear(self.base_model.hidden_dim, n_classes)

    def forward(self, x):
        # Pass input through the ViT model
        x = self.base_model(x)  # Get embeddings directly
        
        # Compute output for each attribute
        outputs = {attr: layer(x) for attr, layer in self.output_layers.items()}
        
        return list(outputs.values())

# Example usage
# Define number of classes for each attribute
num_classes = {attr: train_df_Men_Tshirts[attr].nunique() for attr in attr_columns}

# Initialize the model
model_vit_116 = MultiOutputModelVit(num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_vit_116.parameters(), lr=0.0001)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_L_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_L_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vit_l_16-852ce7e3.pth" to /root/.cache/torch/hub/checkpoints/vit_l_16-852ce7e3.pth
100%|██████████| 1.13G/1.13G [00:05<00:00, 206MB/s] 


In [12]:
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_swin_v2.load_state_dict(torch.load(f"{path_swin_v2}"))
model_swin_v2 = model_swin_v2.to(device)

/tmp/ipykernel_30/3600621464.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_swin_v2.load_state_dict(torch.load(f"{path_swin_v2}"))


In [13]:
torch.manual_seed(42)
model_vit_116.load_state_dict(torch.load(f"{path_vit_l16}"))
model_vit_116 = model_vit_116.to(device)

/tmp/ipykernel_30/3380521888.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_vit_116.load_state_dict(torch.load(f"{path_vit_l16}"))


In [14]:
torch.manual_seed(42)
model_swin_v2.eval()

MultiOutputModelSwin(
  (base_model): SwinTransformer(
    (features): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
        (1): Permute()
        (2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (1): Sequential(
        (0): SwinTransformerBlockV2(
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (attn): ShiftedWindowAttentionV2(
            (qkv): Linear(in_features=128, out_features=384, bias=True)
            (proj): Linear(in_features=128, out_features=128, bias=True)
            (cpb_mlp): Sequential(
              (0): Linear(in_features=2, out_features=512, bias=True)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=512, out_features=4, bias=False)
            )
          )
          (stochastic_depth): StochasticDepth(p=0.0, mode=row)
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (mlp): MLP(
            (

In [15]:
torch.manual_seed(42)
model_vit_116.eval()

MultiOutputModelVit(
  (base_model): VisionTransformer(
    (conv_proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
    (encoder): Encoder(
      (dropout): Dropout(p=0.0, inplace=False)
      (layers): Sequential(
        (encoder_layer_0): EncoderBlock(
          (ln_1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (self_attention): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (dropout): Dropout(p=0.0, inplace=False)
          (ln_2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (mlp): MLPBlock(
            (0): Linear(in_features=1024, out_features=4096, bias=True)
            (1): GELU(approximate='none')
            (2): Dropout(p=0.0, inplace=False)
            (3): Linear(in_features=4096, out_features=1024, bias=True)
            (4): Dropout(p=0.0, inplace=False)
          )
        )
        (encoder_layer_1): EncoderBlock(
 

In [ ]:
test_df = pd.read_csv('../../Dataset/test.csv')
test_df

,id,Category
0,0,Men Tshirts
1,1,Men Tshirts
2,2,Men Tshirts
3,3,Men Tshirts
4,4,Men Tshirts
...,...,...
30200,30484,Women Tops & Tunics
30201,30485,Women Tops & Tunics
30202,30486,Women Tops & Tunics
30203,30487,Women Tops & Tunics


In [ ]:
test_df_Men_Tshirts = test_df[test_df['Category'] == model_category]
test_df_Men_Tshirts['len'] = 5

def format_image_path_test(row):
    return f"../../Dataset/test_images/{str(row['id']).zfill(6)}.jpg"

test_df_Men_Tshirts['image_path'] = test_df_Men_Tshirts.apply(format_image_path_test, axis=1)
test_df_Men_Tshirts

/tmp/ipykernel_30/2539153895.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_Men_Tshirts['len'] = 5
/tmp/ipykernel_30/2539153895.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_Men_Tshirts['image_path'] = test_df_Men_Tshirts.apply(format_image_path_test, axis=1)


,id,Category,len,image_path
0,0,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0000...
1,1,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0000...
2,2,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0000...
3,3,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0000...
4,4,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0000...
...,...,...,...,...
3782,3782,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0037...
3783,3783,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0037...
3784,3784,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0037...
3785,3785,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0037...


In [18]:
# Test dataset without labels
test_dataset = CustomDataset(dataframe=test_df_Men_Tshirts, img_dir='', transform=transform, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [19]:
len(test_loader)

119

In [20]:
from tqdm import tqdm  # Use console version of tqdm

predictions = []

with torch.no_grad():
    for images in tqdm(test_loader):  # This will display in the console
        images = images.cuda() if torch.cuda.is_available() else images

        # Forward pass through both models
        torch.manual_seed(42)
        outputs1 = model_swin_v2(images)
        outputs2 = model_vit_116(images)

        # Initialize a list to hold blended predictions for the batch
        batch_preds = []

        # Loop through the outputs and blend predictions for each attribute
        for out1, out2 in zip(outputs1, outputs2):
            # Blend logits by averaging
            blended_output = (out1 + out2) / 2

            # Get the predicted classes from the blended output
            _, pred = torch.max(blended_output, 1)
            batch_preds.append(pred.cpu().numpy())  # Move to CPU and convert to numpy

        # Stack predictions along a new dimension and add to predictions list
        predictions.append(np.stack(batch_preds, axis=1))  # Shape: (batch_size, num_attributes)

# Combine all predictions into a single array
predictions = np.concatenate(predictions, axis=0)

# Display final predictions
print(predictions)  # This will be an array with shape (num_samples, num_attributes)


100%|██████████| 119/119 [04:02<00:00,  2.04s/it]

[[2 1 0 0 0]
 [3 1 0 2 0]
 [3 1 0 0 1]
 ...
 [1 0 0 0 1]
 [1 1 1 1 1]
 [1 0 1 1 1]]


In [21]:
# Assuming predictions is a numpy array of shape (N, 5)
# Add new columns attr_1 to attr_10 to test_df
for i in range(1, 11):
    test_df_Men_Tshirts[f'attr_{i}'] = np.nan

# Assign predictions to attr_1 to attr_5
test_df_Men_Tshirts[attr_columns] = predictions

# Optionally save the updated test_df to CSV
# test_df.to_csv('test_predictions_with_attrs.csv', index=False)

# Display the first few rows of the updated DataFrame
test_df_Men_Tshirts.head()

/tmp/ipykernel_30/2745049202.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_Men_Tshirts[f'attr_{i}'] = np.nan
/tmp/ipykernel_30/2745049202.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_Men_Tshirts[f'attr_{i}'] = np.nan
/tmp/ipykernel_30/2745049202.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

,id,Category,len,image_path,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
0,0,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0000...,2,1,0,0,0,NaN,NaN,NaN,NaN,NaN
1,1,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0000...,3,1,0,2,0,NaN,NaN,NaN,NaN,NaN
2,2,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0000...,3,1,0,0,1,NaN,NaN,NaN,NaN,NaN
3,3,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0000...,1,0,1,1,1,NaN,NaN,NaN,NaN,NaN
4,4,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0000...,0,1,1,1,0,NaN,NaN,NaN,NaN,NaN


In [22]:
# Inverse transform predictions for each attribute using the stored label encoders
for attr in attr_columns:
    # Inverse transform using the corresponding label encoder
    test_df_Men_Tshirts[attr] = label_encoders[attr].inverse_transform(test_df_Men_Tshirts[attr].astype(int))

# Check the updated DataFrame¯ with original attribute names instead of encoded numbers
test_df_Men_Tshirts

/tmp/ipykernel_30/688606145.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_Men_Tshirts[attr] = label_encoders[attr].inverse_transform(test_df_Men_Tshirts[attr].astype(int))
/tmp/ipykernel_30/688606145.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_Men_Tshirts[attr] = label_encoders[attr].inverse_transform(test_df_Men_Tshirts[attr].astype(int))
/tmp/ipykernel_30/688606145.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

,id,Category,len,image_path,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
0,0,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0000...,multicolor,round,printed,default,long sleeves,NaN,NaN,NaN,NaN,NaN
1,1,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0000...,white,round,printed,typography,long sleeves,NaN,NaN,NaN,NaN,NaN
2,2,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0000...,white,round,printed,default,short sleeves,NaN,NaN,NaN,NaN,NaN
3,3,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0000...,default,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN
4,4,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0000...,black,round,solid,solid,long sleeves,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3782,3782,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0037...,default,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN
3783,3783,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0037...,black,round,printed,default,short sleeves,NaN,NaN,NaN,NaN,NaN
3784,3784,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0037...,default,polo,printed,default,short sleeves,NaN,NaN,NaN,NaN,NaN
3785,3785,Men Tshirts,5,/kaggle/input/visual-taxonomy/test_images/0037...,default,round,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN


In [23]:
# Get value counts for each specified column
columns_of_interest = attr_columns

for column in columns_of_interest:
    print(f"Value counts for {column}:\n")
    print(test_df_Men_Tshirts[column].value_counts(dropna=False))  # Including NaN values
    print("\n" + "-"*50 + "\n")

Value counts for attr_1:

attr_1
default       1420
white         1037
black          703
multicolor     627
Name: count, dtype: int64

--------------------------------------------------

Value counts for attr_2:

attr_2
round    2359
polo     1428
Name: count, dtype: int64

--------------------------------------------------

Value counts for attr_3:

attr_3
printed    2139
solid      1648
Name: count, dtype: int64

--------------------------------------------------

Value counts for attr_4:

attr_4
solid         1648
default       1329
typography     810
Name: count, dtype: int64

--------------------------------------------------

Value counts for attr_5:

attr_5
short sleeves    2925
long sleeves      862
Name: count, dtype: int64

--------------------------------------------------



In [ ]:
sub_df = pd.read_csv('../../Dataset/sample_submission.csv')
sub_df

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
0,47001,Women Tshirts,8,maroon,regular,crop,printed,typography,short sleeves,regular sleeves,default,nu,nu
1,16431,Sarees,10,same as saree\n,temple border,small border,cream,party,jacquard,woven design,zari woven,ethnic motif,no
2,55700,Women Tops & Tunics,10,white,fitted,regular,round neck,casual,solid,solid,short sleeves,regular sleeves,knitted
3,15698,Sarees,10,same as saree\n,zari,small border,white,party,jacquard,woven design\n,zari woven,floral,no
4,30330,Kurtis,9,yellow,a-line,knee length\n,daily,net,default,solid,three-quarter sleeves,regular,nu


In [ ]:
sub_df_Men_Tshirts = test_df_Men_Tshirts.drop('image_path', axis = 1)
# sub_df_Men_Tshirts.replace("unknown", np.nan, inplace=True)
sub_df_Men_Tshirts.to_csv(f"output/sub_df_{model_category}_swim_0.5_vit_0.5_b5_blend.csv", index = False)
sub_df_Men_Tshirts.to_csv(f"sub_df_{model_category}_swim_0.5_vit_0.5_b5_blend.csv", index = False)

sub_df_Men_Tshirts

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
0,0,Men Tshirts,5,multicolor,round,printed,default,long sleeves,NaN,NaN,NaN,NaN,NaN
1,1,Men Tshirts,5,white,round,printed,typography,long sleeves,NaN,NaN,NaN,NaN,NaN
2,2,Men Tshirts,5,white,round,printed,default,short sleeves,NaN,NaN,NaN,NaN,NaN
3,3,Men Tshirts,5,default,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN
4,4,Men Tshirts,5,black,round,solid,solid,long sleeves,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3782,3782,Men Tshirts,5,default,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN
3783,3783,Men Tshirts,5,black,round,printed,default,short sleeves,NaN,NaN,NaN,NaN,NaN
3784,3784,Men Tshirts,5,default,polo,printed,default,short sleeves,NaN,NaN,NaN,NaN,NaN
3785,3785,Men Tshirts,5,default,round,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN
